## Experimental Design
In a closed room with no open windows or doors, I set up an oscillating fan blowing into the center of the room. The fan is a tower style home fan that is much taller than it is wide. 

I tied a string to the center of the fan and allowed the fan to oscillate all the way to the right. Here I placed one of the windLog instruments (FunStorm) as close as I could to the fan, which was sitting atop my dresser. I tied a knot in the string where the front of the wind instrument was placed. I did this so I could place the other two sensors nearly the same distance away from the fan. 

I allowed the fan to oscillate to the far left, where I placed the second windLog insturment (LlamaLog), being careful to align the instrument so that is the same distance away as FunStorm using the string. I completed the placement by putting the kestrel in the center using the string. 

The length of string that was used is approximately 0.40 m. The height of the sensors was 1.65 m. 

The fan was set to full speed, and the door was closed. The trial period began at 11:35 AM on July 30th, 2023. The trial ended sometime around 5:30 PM the same day. The fan was turned off, but data collection continued while I wrote this and downloaded the data.

All sensors were set to record wind measurements every minute. Both wind log meters were free of any cables during the data collection period. 

Data was downloaded from the windLog instruments using the default software on my work laptop (Windows). At the time of download, Funstorm had 4.6V reading, and LlamaLog had 4.3V.

Curiously, The Funstorm instrument recording a spike in wind measurements when I connected to the computer. This was mirrored in previous experiments. 